In [1]:
#!pip install facebook-sdk
import json
import facebook
import requests
import pandas as pd
import numpy as np
import unicodedata
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Function which takes in the aguement page name and returns 25 recent posts on a given page

In [14]:
def getPosts(pageName):
    if pageName == 'NEWJ':
        token = 'EAAwZCu1v8wlQBAI6kdPJNZBxxu9BwO1mLI2A21IGvuB5OKnPWQREzCgt4nfkSgfdEiQnsDoNpbk4PuKlg234fNKgw1x46meVKZCtpZCiHtosO9NpwuZBs1ugTlj21BcJQpXbK9FSjZARp34q9DvxujCnCEd0AOUwNECyZAbgck1NgDtHmZAWD4yLTi7k8K0ZBSMkZD'
        page_id = '100589847991303' 
    elif pageName == 'Newj plus':
        token = 'EAAlgYmp3afkBAKHR5oZBZChW1AVNW2h17n7W3lbrIMuQ0x6NqqLMMmNZCnGlHZCKoyKseF1quuSZAKJgnLZBIJiCLwVFHRuiLTcIXwJXouOZBrTVTXRz1IOcxJfk7tNAJ3Fzw9YEpRxOlPkiiZCntjl25MPmspi4gCtMZBHPY4iPbFzIi9Juba8SCxrZBNkvJJQzsZD'
        page_id = '1954072204870360' 
    else:
        print("Invalid Choice")
    api = facebook.GraphAPI(token)  
    posts = api.get_connections(id=page_id,connection_name='posts',limit=10)
    paging = True      
    while paging:
            if 'paging' in posts:
                x = posts['paging']
                if 'next' in posts:
                    url = posts['next']
                    posts = requests.get(url).json()
                else:
                    paging = False
            else:
                    paging = False
    return posts
   

In [15]:
recent1=getPosts("NEWJ")
recent1

{'data': [{'created_time': '2020-09-26T15:05:00+0000',
   'message': "A Son's Love-Mysuru Man Takes His Mother On A Nationwide 56,000 km Yatra",
   'id': '100589847991303_369366994446919'},
  {'created_time': '2020-09-26T11:09:51+0000',
   'message': 'Is Pakistan suppressing the voices of PoK to please China? Gilgit Baltistan angers Opposition leaders in Pakistan',
   'id': '100589847991303_369247297792222'},
  {'created_time': '2020-09-26T09:38:13+0000',
   'message': 'Internet Pioneer Dr Vint Cerf warns of a ‘digital dark age’',
   'id': '100589847991303_369198771130408'},
  {'created_time': '2020-09-26T07:28:18+0000',
   'message': 'Saree, sneakers and hula hoops, a sensational performance that has gone viral\nEshna Kutty',
   'id': '100589847991303_369134621136823'},
  {'created_time': '2020-09-26T03:30:01+0000',
   'message': 'Dating back to the 1800s, Garos are a Tibeto-Burman ethnic group in Meghalaya',
   'id': '100589847991303_368593527857599'},
  {'created_time': '2020-09-25T

### Function which takes the object returned by the getPosts() function and returns comments on the 25 posts

In [22]:
def getComments(post):
    comments_df = []
    token = 'EAAwZCu1v8wlQBAI6kdPJNZBxxu9BwO1mLI2A21IGvuB5OKnPWQREzCgt4nfkSgfdEiQnsDoNpbk4PuKlg234fNKgw1x46meVKZCtpZCiHtosO9NpwuZBs1ugTlj21BcJQpXbK9FSjZARp34q9DvxujCnCEd0AOUwNECyZAbgck1NgDtHmZAWD4yLTi7k8K0ZBSMkZD'
    api = facebook.GraphAPI(token)
    for i in post['data']:
        temp = dict()
        comms = api.get_connections(id=i['id'], connection_name='comments', summary = 'TRUE')
        count = 0
        allcoms = {}
        paging = True
        allcoms[count] = comms['data'].copy()
        count+=1
        while paging:
            if 'paging' in comms:
                x = comms['paging']
                if 'next' in x:
                    url = x['next']
                    comms = requests.get(url).json()
                    allcoms[count] = comms['data'].copy()
                    count+=1
                else:
                    paging = False
            else:
                paging = False
        if('message' in i.keys()):
            temp['post_id'] = i['id']
            temp['video_title'] = i['message']
        #temp['video_title'] = i['story']
            temp['posted_date'] = i['created_time']
        allcomments = []
        for element in allcoms:
            entry = allcoms[element]
            for j in entry:
                allcomments.append(j['message'])
        temp['comments_text'] = allcomments
        temp['comments_count'] = len(allcomments)
        comments_df.append(temp.copy())    
    pd.set_option('display.max_colwidth', 100)
    return(pd.DataFrame(comments_df))

In [23]:
#recent1 = getPosts('English NEWJ')
inputdf1 = getComments(recent1)
inputdf1['comments'] = inputdf1['comments_text'].apply(lambda x: ','.join(map(str, x)))
inputdf1

GraphAPIError: (#100) Pages Public Content Access requires either app secret proof or an app token

In [16]:
inputdf1.to_excel("newjcomments1.xlsx")

NameError: name 'inputdf1' is not defined

### Importing the required libraries for the Sentiment Analysis

In [8]:
# importing for ipywidgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [9]:
#import to filter warnings
import warnings
warnings.filterwarnings('ignore')
#!pip install numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from matplotlib.pyplot import xticks
%matplotlib inline
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(16,10)})
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', -1)
#!pip install wordcloud
#!pip install -U textblob

In [10]:
import calendar
import datetime as dt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('movie_reviews')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from nltk.corpus import movie_reviews

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [11]:
neg=[ 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't","not","don't","dont","do not","won't"]

In [12]:
eng_stopwords = stopwords.words('english')
eng_stopwords = [x for x in eng_stopwords if x not in neg]

### Function for translation

In [13]:
from googletrans import Translator
translator = Translator()

In [16]:
translations_list=[]
def translatorfunc(comm_df):
    for i in comm_df.comments_text:
        translations = translator.translate(i,dest='en')
        tr_list=[]
        for i in translations:
            tr_list.append(i.text)
        translations_list.append(tr_list)
    inputdf1["translated_comments"]=translations_list
    return comm_df

In [17]:
translatorfunc(inputdf1)

post_id  \
0  1954072204870360_2775249392752633   
1  1954072204870360_663462111255440    
2  1954072204870360_2775671979377041   
3  1954072204870360_2775103769433862   
4  1954072204870360_2775232886087617   
5  1954072204870360_2775156489428590   
6  1954072204870360_2775112809432958   
7  1954072204870360_2775034842774088   
8  1954072204870360_2774990362778536   
9  1954072204870360_2774943979449841   

                                                                                                                                                           video_title  \
0  क्या आपकी जेब में ऐसा नोट है जिसका पहला और आखिरी अंक 9 है? अगर हां, तो जानिए सोशल मीडिया पर क्यों हो रही है ऐसे नोट की चर्चा?\n\n(via NEWJ Dharohar)                  
1  आईआईटी गुवाहाटी के कार्यक्रम से पीएम नरेंद्र मोदी LIVE                                                                                                                
2  श्री वेंकटेश्वर मंदिर में गैर-हिंदुओं के प्रवेश को लेकर क्या हैं पारंपरिक नियम? सीएम वाईएस जगनमोहन रेड्डी के दर्शन को लेकर क्यों हो रहा है विवाद? जानिए पूरी कहानी?   
3  टारगेट पूरा करने के लिए इस हॉस्पिटल में हुई फर्जी सैंपलिंग, डॉक्टर के फर्जीवाड़े का वीडियो वायरल                                                                      
4  राज्यसभा में सरेआम गुंडगर्दी करने वाले माननीय सांसदो की हरकतो पर हम पूछते हैं ये तमाम सवाल                                                                            
5  नेता ही नहीं कृषि कानून को लेकर अब आमने-सामने आए पंजाबी इंडस्ट्री के सितारे, किसी ने किया समर्थन तो किसी ने बताया गलत                                                 
6  इस बार पकिस्तान ही नहीं, भारत ने तुर्की को भी संयुक्त राष्ट्र की सभा में दिया करारा जवाब                                                                              
7  दिल्ली में त्यौहारों के दौरान धमाके की बना रहे थे योजना. एनआईए ने पश्चिम बंगाल और केरल से अलकायदा के 9 आतंकियों को धरदबोचा                                            
8  ‘डर लगता तो फौज में नहीं भेजते अपने बेटे’ चीनी सीमा पर तैनात अपने बेटों पर इन लद्दाखी परिवारों को है गर्व\n\nADGPI - Indian Army                                      
9  विरोध करने के चक्कर में सदन की मर्यादा भूले विपक्ष के नेता, किसी ने फाड़ी रुल बुक तो किसी ने की धक्का मुक्की                                                          

                posted_date  \
0  2020-09-22T07:35:01+0000   
1  2020-09-22T06:46:19+0000   
2  2020-09-22T05:05:00+0000   
3  2020-09-22T02:35:00+0000   
4  2020-09-21T16:05:00+0000   
5  2020-09-21T14:35:00+0000   
6  2020-09-21T13:10:00+0000   
7  2020-09-21T11:35:00+0000   
8  2020-09-21T10:05:00+0000   
9  2020-09-21T08:30:00+0000   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [25]:
inputdf1[["comments_text","translated_comments"]].head()

comments_text  \
0  []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [18]:
inputdf1[["comments_text","translated_comments"]]

comments_text  \
0  []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [19]:
inputdf1['comments_text_translated'] = inputdf1.translated_comments.apply(lambda x: ','.join(map(str, x)))
#inputdf1[['comments_text_translated','video_title']]

### Function for sentiment

In [21]:
def get_Sentiment(video_data):
    titles=[]
    sentiment_on_video=[]
    subjectivity_on_video=[]
    sentim=[]
    subjec=[]
    postId=[]
    clas=[]
    pos=[]
    neg=[]
   
        
    for video in video_data.video_title:
        #videodata_final=video_data[video_data.comments_count>20]
        comments_words = video_data.loc[video_data['video_title'] == video].comments_text_translated.str.lower().str.cat(sep=' ').split()
        #comments_as_text_block=" ".join(comments_words)
        #tokenizer = RegexpTokenizer(r'\w+')
        #result = tokenizer.tokenize(comments_as_text_block)
        #text = " ".join(result)
        #tokens_stopwords = word_tokenize(text)
        no_stopwords_tokens = [word for word in comments_words if not word in stopwords.words()]
    #print(no_stopwords_tokens)
    #print(text_tokens)
        textblock_without_stopwords=" ".join(no_stopwords_tokens)
        #textblock_with_stopwords=" ".join(tokens_stopwords)
        sent          = TextBlob(textblock_without_stopwords)
        polarity      = sent.sentiment.polarity
        subjectivity  = sent.sentiment.subjectivity
        #sent          = TextBlob(text, analyzer = NaiveBayesAnalyzer())
        #classification= sent.sentiment.classification
        #positive      = sent.sentiment.p_pos
        #negative      = sent.sentiment.p_neg
        if (polarity <0.1 and polarity >= 0):
            Senti="Neutral"
        elif (polarity >= 0.1 and polarity<0.4):
            Senti=" Slightly Positive"
        elif (polarity >= 0.4 and polarity<=0.5):
            Senti="Positive"
        elif (polarity > 0.5):
            Senti=" Highly Positive"
        elif (polarity <= -0.3):
            Senti=" Highly Negative"
        else:
            Senti="Negative"
        if (subjectivity <0.2):
            Subj="Objective"
        elif (subjectivity >0.2 and subjectivity >= 0.4):
            Subj="Slightly Subjective"
        elif (subjectivity >0.4 and subjectivity >= 0.6):
            Subj="Subjective"
        else:
            Subj="Highly Subjective"

        subjec.append(subjectivity)
        sentim.append(polarity)
        subjectivity_on_video.append(Subj)
        sentiment_on_video.append(Senti)
        #clas.append(classification)
        #pos.append(positive)
        #neg.append(negative)
    for title in video_data.video_title:
        titles.append(title)
    
    for i in video_data.post_id:
        postId.append(i)
    global Sentiment
    data={'post_Id': postId,'Video_Title': titles, 'Sentiment on Video': sentiment_on_video, 
          'Subjectivity on Video': subjectivity_on_video,'Sentiment_Value':sentim,
          'Subjectivity value':subjec}#'Classification':classification, 'postive_value':positive,'negative_value':negative}
    Sentiment=pd.DataFrame(data)
    return Sentiment
    

In [22]:
inputdf1[["video_title","comments_count"]]

,video_title,comments_count
0,"क्या आपकी जेब में ऐसा नोट है जिसका पहला और आखिरी अंक 9 है? अगर हां, तो जानिए सोशल मीडिया पर क्यों हो रही है ऐसे नोट की चर्चा?\n\n(via NEWJ Dharohar)",0
1,आईआईटी गुवाहाटी के कार्यक्रम से पीएम नरेंद्र मोदी LIVE,68
2,श्री वेंकटेश्वर मंदिर में गैर-हिंदुओं के प्रवेश को लेकर क्या हैं पारंपरिक नियम? सीएम वाईएस जगनमोहन रेड्डी के दर्शन को लेकर क्यों हो रहा है विवाद? जानिए पूरी कहानी?,16
3,"टारगेट पूरा करने के लिए इस हॉस्पिटल में हुई फर्जी सैंपलिंग, डॉक्टर के फर्जीवाड़े का वीडियो वायरल",30
4,राज्यसभा में सरेआम गुंडगर्दी करने वाले माननीय सांसदो की हरकतो पर हम पूछते हैं ये तमाम सवाल,132
5,"नेता ही नहीं कृषि कानून को लेकर अब आमने-सामने आए पंजाबी इंडस्ट्री के सितारे, किसी ने किया समर्थन तो किसी ने बताया गलत",17
6,"इस बार पकिस्तान ही नहीं, भारत ने तुर्की को भी संयुक्त राष्ट्र की सभा में दिया करारा जवाब",16
7,दिल्ली में त्यौहारों के दौरान धमाके की बना रहे थे योजना. एनआईए ने पश्चिम बंगाल और केरल से अलकायदा के 9 आतंकियों को धरदबोचा,80
8,‘डर लगता तो फौज में नहीं भेजते अपने बेटे’ चीनी सीमा पर तैनात अपने बेटों पर इन लद्दाखी परिवारों को है गर्व\n\nADGPI - Indian Army,54
9,"विरोध करने के चक्कर में सदन की मर्यादा भूले विपक्ष के नेता, किसी ने फाड़ी रुल बुक तो किसी ने की धक्का मुक्की",588


In [20]:
inputdf_final=inputdf1[inputdf1.comments_count>30]
inputdf_final[["video_title","comments_count"]]

,video_title,comments_count
1,आईआईटी गुवाहाटी के कार्यक्रम से पीएम नरेंद्र मोदी LIVE,68
4,राज्यसभा में सरेआम गुंडगर्दी करने वाले माननीय सांसदो की हरकतो पर हम पूछते हैं ये तमाम सवाल,132
7,दिल्ली में त्यौहारों के दौरान धमाके की बना रहे थे योजना. एनआईए ने पश्चिम बंगाल और केरल से अलकायदा के 9 आतंकियों को धरदबोचा,80
8,‘डर लगता तो फौज में नहीं भेजते अपने बेटे’ चीनी सीमा पर तैनात अपने बेटों पर इन लद्दाखी परिवारों को है गर्व\n\nADGPI - Indian Army,54
9,"विरोध करने के चक्कर में सदन की मर्यादा भूले विपक्ष के नेता, किसी ने फाड़ी रुल बुक तो किसी ने की धक्का मुक्की",588


In [26]:
get_Sentiment(inputdf_final)

,post_Id,Video_Title,Sentiment on Video,Subjectivity on Video,Sentiment_Value,Subjectivity value
0,1954072204870360_663462111255440,आईआईटी गुवाहाटी के कार्यक्रम से पीएम नरेंद्र मोदी LIVE,Neutral,Slightly Subjective,0.013258,0.427418
1,1954072204870360_2775232886087617,राज्यसभा में सरेआम गुंडगर्दी करने वाले माननीय सांसदो की हरकतो पर हम पूछते हैं ये तमाम सवाल,Neutral,Slightly Subjective,0.046843,0.422815
2,1954072204870360_2775034842774088,दिल्ली में त्यौहारों के दौरान धमाके की बना रहे थे योजना. एनआईए ने पश्चिम बंगाल और केरल से अलकायदा के 9 आतंकियों को धरदबोचा,Slightly Positive,Slightly Subjective,0.171884,0.532942
3,1954072204870360_2774990362778536,‘डर लगता तो फौज में नहीं भेजते अपने बेटे’ चीनी सीमा पर तैनात अपने बेटों पर इन लद्दाखी परिवारों को है गर्व\n\nADGPI - Indian Army,Slightly Positive,Slightly Subjective,0.362500,0.575000
4,1954072204870360_2774943979449841,"विरोध करने के चक्कर में सदन की मर्यादा भूले विपक्ष के नेता, किसी ने फाड़ी रुल बुक तो किसी ने की धक्का मुक्की",Neutral,Slightly Subjective,0.063050,0.436405
